## Import all the necessary packages

In [1]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

Using TensorFlow backend.


In [2]:
from tqdm import tqdm
tqdm().pandas()

0it [00:00, ?it/s]


## Preprocessing the data

In [3]:
# Loading a text file into memory
def load_doc(fn):
    file = open(fn, 'r')
    text = file.read()
    file.close()
    return text

# get all imgs with their captions
def captions(fn):
    file = load_doc(fn)
    cap = file.split('\n')
    d ={}
    for i in cap[:-1]:
        img, i = i.split('\t')
        if img[:-2] not in d:
            d[img[:-2]] = [ i ]
        else:
            d[img[:-2]].append(i)
    return d

#Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning(text):
    table = str.maketrans('','',string.punctuation)
    for img,caps in text.items():
        for i,img_caption in enumerate(caps):
            img_caption.replace("-"," ")
            desc = img_caption.split()
           
            #converts to lowercase
            desc = [word.lower() for word in desc]
            
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            
            #convert back to string
            img_caption = ' '.join(desc)
            text[img][i]= img_caption
    return text

# build vocabulary of all unique words
def text_vocabulary(descriptions):
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

#All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [4]:
# Set the path
dataset_text = "Flickr8k_text"
dataset_images = "Flickr8k_Dataset"

# Prepare the text data
fn = dataset_text + "/" + "Flickr8k.token.txt"

In [5]:
# Loading file containing data and mapping them into dictionary
desc = captions(fn)
print("Length of descriptions = " ,len(desc))

Length of descriptions =  8092


In [6]:
# cleaning the text
clean = cleaning(desc)

In [7]:
len(clean)

8092

In [8]:
# building vocab
vocab = text_vocabulary(clean)
print("Length of vocab = " , len(vocab))

Length of vocab =  8763


In [9]:
#save_descriptions(clean, "descriptions.txt")

## Extracting the feature vector from images

In [10]:
# extract features for all images and we will map image names with their respective feature array.
def extract_features(d):
    model = Xception(include_top=False, pooling='avg')
    features = {}
    for img in tqdm(os.listdir(d)):
        fn = d + "/" + img
        image = Image.open(fn)
        image = image.resize((299,299))
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image-1.0
        feature = model.predict(image)
        features[img] = feature
    return features

In [11]:
# extract features and then dump dictionary into a pickle file
#features = extract_features(dataset_images)
#dump(features, open("features.p","wb"))

In [12]:
features = load(open("features.p","rb"))

## Loading dataset for Training the Model

In [13]:
# load the data
def load_imgs(fn):
    file = load_doc(fn)
    imgs = file.split("\n")[:-1]
    return imgs

In [14]:
# Create dictionary that contains captions for each image
def descriptions(fn, imgs):
    file = load_doc(fn)
    desc = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words) < 1:
            continue
        img, ic = words[0], words[1:]
        
        if img in imgs:
            if img not in desc:
                desc[img] = []
            d = '<start> ' + " ".join(ic) + ' <end>'
            desc[img].append(d)
    
    return desc

In [15]:
# load dictionary for image names and their feature vector which we have previously 
# extracted from the Xception model
def load_features(imgs):
    features = load(open("features.p", "rb"))
    f = {k:features[k] for k in imgs}
    return f

In [16]:
fn = dataset_text + "/" + "Flickr_8k.trainImages.txt"

In [17]:
train_imgs = load_imgs(fn)
train_descriptions = descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

## Tokenizing the Vocabulary

In [18]:
# converting dictionary to clean list of captions
def to_list(cap):
    all_cap = []
    for key in cap.keys():
        [all_cap.append(i) for i in cap[key]]
    return all_cap

In [19]:
from keras.preprocessing.text import Tokenizer

# using tokenizer class, we will represent each token with an integer
def tokenize(cap):
    l = to_list(cap)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(l)
    return tokenizer

In [20]:
# give each word an index, and store that into tokenizer.p pickle file
tokenizer = tokenize(train_descriptions)
# dump(tokenizer, open('tokenizer.p', 'wb'))
size = len(tokenizer.word_index) + 1
size

7577

In [21]:
# calculate maximum length of captions
def maximum_length(captions):
    l = to_list(captions)
    return max(len(i.split()) for i in l)

In [22]:
max_length = maximum_length(desc)
max_length

32

## Data Generator

In [23]:
def create_sequences(tokenizer, length, desc_list, feature):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=size)[0]
            # store
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

In [24]:
# data generator, used by model.fit_generator()
def tgenerator(captions, features, tokenizer, length):
    while 1:
        for key, l in captions.items():
            feature = features[key][0]
            img, seq, word = create_sequences(tokenizer, length, l, feature)
            yield [[img, seq], word]

In [25]:
[a,b],c = next(tgenerator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape

((47, 2048), (47, 32), (47, 7577))

## Defining the CNN-RNN model

In [26]:
from keras.utils import plot_model

In [27]:
# define the captioning model
def define_model(vocab_size, max_length):
    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
    return model

## Training the model

In [28]:
#train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', size)
print('Description Length: ', max_length)

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 7577
Description Length:  32


In [29]:
model = define_model(size, max_length)
epochs = 10
steps = len(train_descriptions)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 256)      1939712     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [31]:
os.mkdir("models")
for i in range(epochs):
    generator = tgenerator(train_descriptions, train_features, tokenizer, max_length)
    model.fit(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save("models/model_" + str(i) + ".h5")

C:\Users\daani\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
6000/6000 [==============================] - 2241s 374ms/step - loss: 4.4796
Epoch 1/1
6000/6000 [==============================] - 1923s 321ms/step - loss: 3.6431
Epoch 1/1
6000/6000 [==============================] - 1850s 308ms/step - loss: 3.3575
Epoch 1/1
6000/6000 [==============================] - 2051s 342ms/step - loss: 3.1840
Epoch 1/1
6000/6000 [==============================] - 2085s 347ms/step - loss: 3.0642
Epoch 1/1
6000/6000 [==============================] - 2250s 375ms/step - loss: 2.9743
Epoch 1/1
6000/6000 [==============================] - 2403s 400ms/step - loss: 2.9086
Epoch 1/1
6000/6000 [==============================] - 2346s 391ms/step - loss: 2.8536
Epoch 1/1
6000/6000 [==============================] - 2215s 369ms/step - loss: 2.8132
Epoch 1/1
6000/6000 [==============================] - 2095s 349ms/step - loss: 2.7779
